In [145]:
import pandas as pd
import numpy as np
import re
import warnings 
warnings.filterwarnings('ignore')

# Importing Data set

In [146]:
data_train=pd.read_csv('Data\Dataset_emp\Train.csv')
data_test=pd.read_csv('Data\Dataset_emp\Test.csv')

In [147]:
list(data_train)

['Employee_ID',
 'Gender',
 'Age',
 'Education_Level',
 'Relationship_Status',
 'Hometown',
 'Unit',
 'Decision_skill_possess',
 'Time_of_service',
 'Time_since_promotion',
 'growth_rate',
 'Travel_Rate',
 'Post_Level',
 'Pay_Scale',
 'Compensation_and_Benefits',
 'Work_Life_balance',
 'VAR1',
 'VAR2',
 'VAR3',
 'VAR4',
 'VAR5',
 'VAR6',
 'VAR7',
 'Attrition_rate']

In [148]:
data_test['Attrition_rate'] = np.nan
data_train['data'] = 'train' 
data_test['data'] = 'test'
data_test = data_test[data_train.columns]
data_all = pd.concat([data_train,data_test],axis=0)

In [149]:
data_all

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,...,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate,data
0,EID_23371,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,...,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841,train
1,EID_18000,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,...,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670,train
2,EID_3891,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,...,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851,train
3,EID_17492,F,26.0,3,Single,Lebanon,Human Resource Management,Behavioral,4.0,3,...,1.0,3,-1.8176,-0.4537,NaN,3,7,3,0.0668,train
4,EID_22534,F,31.0,1,Married,Springfield,Logistics,Conceptual,5.0,4,...,3.0,1,0.7516,-0.4537,2.0,2,8,2,0.1827,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,EID_22547,F,32.0,3,Single,Franklin,Sales,Directive,3.0,3,...,2.0,1,-0.1048,0.7075,1.0,4,7,4,NaN,test
2996,EID_10066,F,NaN,2,Single,Franklin,Marketing,Conceptual,10.0,2,...,1.0,3,0.7516,0.7075,3.0,4,8,3,NaN,test
2997,EID_7126,F,60.0,4,Single,Lebanon,Logistics,Analytical,35.0,1,...,3.0,3,-1.8176,-0.4537,NaN,4,8,4,NaN,test
2998,EID_4929,F,51.0,1,Married,Springfield,IT,Behavioral,23.0,1,...,1.0,5,0.7516,-0.4537,2.0,3,9,2,NaN,test


# Imputing Missing Values

In [150]:
data_all.isnull().sum()

Employee_ID                     0
Gender                          0
Age                           573
Education_Level                 0
Relationship_Status             0
Hometown                        0
Unit                            0
Decision_skill_possess          0
Time_of_service               196
Time_since_promotion            0
growth_rate                     0
Travel_Rate                     0
Post_Level                      0
Pay_Scale                      12
Compensation_and_Benefits       0
Work_Life_balance              16
VAR1                            0
VAR2                          794
VAR3                            0
VAR4                          954
VAR5                            0
VAR6                            0
VAR7                            0
Attrition_rate               3000
data                            0
dtype: int64

In [151]:
null_cols=data_all.isnull().sum()[data_all.isnull().sum()>0].keys()
null_cols=null_cols.drop(["Attrition_rate"],1)
null_cols

Index(['Age', 'Time_of_service', 'Pay_Scale', 'Work_Life_balance', 'VAR2',
       'VAR4'],
      dtype='object')

In [152]:
data_all['Age'].fillna(data_all['Age'].median(), inplace=True)
data_all['Time_of_service'].fillna(data_all['Time_of_service'].median(), inplace=True)
data_all['Pay_Scale'].fillna(data_all['Pay_Scale'].median(), inplace=True)
data_all['Work_Life_balance'].fillna(data_all['Work_Life_balance'].median(), inplace=True)
data_all['VAR2'].fillna(data_all['VAR2'].median(), inplace=True)
data_all['VAR4'].fillna(data_all['VAR4'].median(), inplace=True)
data_all=data_all.drop(["Employee_ID"],1)

In [153]:
data_all['Age'] = pd.cut(x=data_all['Age'], bins=[20,30,40,50,60])
#data_all['Time_of_service'] = pd.cut(x=data_all['Time_of_service'], bins=[10,20,30,40,50])
#data_all['growth_rate'] = pd.cut(x=data_all['growth_rate'], bins=[20,30,40,50,60,70,80])

# Creating Dummies

In [154]:
##CReating dummies for categorical vars
def dummies(data,var,freq_cutoff=0):
    t= data[var].value_counts(normalize=True) 
    t=t[t.values>freq_cutoff] 
    t=t.sort_values() 
    t_min=t.idxmin()
    t=t.drop([t_min])
    categories=t.index

    for cat in categories :
        name=var+'_'+cat
        name=re.sub(" ","",name) 
        name=re.sub("-","_",name)
        name=re.sub("\\?","Q",name) 
        name=re.sub("<","LT_",name)
        name=re.sub("\\+","",name) 
        name=re.sub("\\/","_",name) 
        name=re.sub(">","GT_",name) 
        name=re.sub("=","EQ_",name)
        name=re.sub(",","",name)
        data[name]=(data[var]==cat)+0 
               
    data=data.drop(columns=[var])
    return data

In [155]:
for i in data_all:
    if(i!='Attrition_rate' and i!='data'):
        data_all[i] = data_all[i].astype(str)
        data_all = dummies(data_all,i)

In [156]:
list(data_all)

['Attrition_rate',
 'data',
 'Gender_F',
 'Age_(40.050.0]',
 'Age_(50.060.0]',
 'Age_(30.040.0]',
 'Age_(20.030.0]',
 'Education_Level_1',
 'Education_Level_2',
 'Education_Level_4',
 'Education_Level_3',
 'Relationship_Status_Married',
 'Hometown_Washington',
 'Hometown_Franklin',
 'Hometown_Springfield',
 'Hometown_Lebanon',
 'Unit_Quality',
 'Unit_Production',
 'Unit_Marketing',
 'Unit_HumanResourceManagement',
 'Unit_AccountingandFinance',
 'Unit_Purchasing',
 'Unit_R&D',
 'Unit_Operarions',
 'Unit_Sales',
 'Unit_Logistics',
 'Unit_IT',
 'Decision_skill_possess_Directive',
 'Decision_skill_possess_Analytical',
 'Decision_skill_possess_Behavioral',
 'Time_of_service_42.0',
 'Time_of_service_41.0',
 'Time_of_service_40.0',
 'Time_of_service_39.0',
 'Time_of_service_38.0',
 'Time_of_service_37.0',
 'Time_of_service_36.0',
 'Time_of_service_35.0',
 'Time_of_service_34.0',
 'Time_of_service_33.0',
 'Time_of_service_32.0',
 'Time_of_service_31.0',
 'Time_of_service_29.0',
 'Time_of_servi

# splitting back in train and test 

In [157]:
data_train=data_all[data_all['data']=='train'] 
del data_train['data'] 
data_test=data_all[data_all['data']=='test']
data_test.drop(['Attrition_rate','data'],axis=1,inplace=True) 

# splitting train in train1 and train2 

In [190]:
from sklearn.model_selection import train_test_split
train1, train2 = train_test_split(data_train, test_size = 0.25,random_state=2)

x_train1=train1.drop(["Attrition_rate"],1)
y_train1=train1["Attrition_rate"]

x_train2=train2.drop(["Attrition_rate"],1)
y_train2=train2["Attrition_rate"]

x_train1.reset_index(drop=True,inplace=True)
y_train1.reset_index(drop=True,inplace=True)

In [191]:
from sklearn.svm import SVR
svr = SVR(kernel='rbf', C=1.0)
svr.fit(x_train1, y_train1)

SVR(C=0.01, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='sigmoid', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [192]:
y_pred1 = svr.predict(x_train1)

In [193]:
y_pred2 = svr.predict(x_train2)

In [194]:
score1=100*max(0,1-np.sqrt(((y_train1 - y_pred1) ** 2).mean()))
score2=100*max(0,1-np.sqrt(((y_train2 - y_pred2) ** 2).mean()))

In [195]:
score1

81.03338239983096

In [196]:
score2

81.59828719859365

In [197]:
y_test_pred = svr.predict(data_test)

In [198]:
pd.DataFrame(y_test_pred).to_csv("laveena_valecha_project_emp_svc.csv",header="Attrition_rate",index=False)